In [29]:
import re
import json
import nltk
import string
import random
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model


nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khalf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
# read json file
with open('C:/Users/khalf/Code/LegalScope/LegalScopeChatbot/intents.json', 'r', encoding='utf-8') as content:
    chatbot = json.load(content)

#convert to dataframe  
data = pd.DataFrame(chatbot['intents'])

print(data.head())

     tag                                           patterns  \
0  qna 1  [Apa tujuan utama dari pembentukan Kitab Undan...   
1  qna 2  [Apa yang dimaksud dengan Kitab Undang-Undang ...   
2  qna 3  [Bagaimana ruang lingkup berlakunya ketentuan ...   
3  qna 4  [Apa yang dimaksud dengan asas wilayah atau te...   
4  qna 5  [Bagaimana ketentuan Pasal 2 ayat (1) dan (2) ...   

                                           responses context  
0  [Tujuan utama dari pembentukan Kitab Undang-Un...      []  
1  [Kitab Undang-Undang Hukum Pidana (KUHP) adala...      []  
2  [Ketentuan peraturan Perundang-undangan hukum ...      []  
3  [Asas wilayah atau teritorial adalah asas hubu...      []  
4  [Hukum yang hidup dalam masyarakat yang terdap...      []  


In [18]:
#dictionary 
dict = {"tag": [], "patterns": [], "responses": []}

#iterate trough the data
for i in range (len(data)):
  pattern = data[data.index==i]['patterns'].values[0]
  response = data[data.index==i]['responses'].values[0]
  tag = data[data.index==i]['tag'].values[0]
  for j in range (len(pattern)):
    dict['tag'].append(tag)
    dict['patterns'].append((pattern[j]))
    dict['responses'].append(response)

#showing the data    
data = pd.DataFrame.from_dict(dict)
data

,tag,patterns,responses
0,qna 1,Apa tujuan utama dari pembentukan Kitab Undang...,[Tujuan utama dari pembentukan Kitab Undang-Un...
1,qna 1,Mengapa diperlukan pembentukan Kitab Undang-Un...,[Tujuan utama dari pembentukan Kitab Undang-Un...
2,qna 1,Apa alasan utama di balik revisi Kitab Undang-...,[Tujuan utama dari pembentukan Kitab Undang-Un...
3,qna 1,Apa yang mendorong pembaruan Kitab Undang-Unda...,[Tujuan utama dari pembentukan Kitab Undang-Un...
4,qna 1,Apa motivasi utama dalam menyusun KUHP yang baru?,[Tujuan utama dari pembentukan Kitab Undang-Un...
...,...,...,...
66,qna 46,Bagaimana ketentuan mengenai tindak pidana kor...,[Ketentuan tindak pidana korupsi (TIpikor) men...
67,qna 47,Bagaimana KUHP mengatur tindak pidana narkotika?,[Tindak pidana narkotika diatur dari Pasal 609...
68,qna 48,Bagaimana KUHP ini mengatur tindak pidana perd...,[Ketentuan mengenai larangan perdagangan orang...
69,qna 49,Bagaimana KUHP ini mengatur tindak pidana penc...,[Kitab Undang-Undang Hukum Pidana (KUHP) Indon...


In [22]:
with open('C:/Users/khalf/Code/LegalScope/LegalScopeChatbot/intents.json', 'r', encoding='utf-8') as content:
    val_chatbot = json.load(content)
val_data = pd.DataFrame(val_chatbot['intents'])

In [23]:
#dictionaries
dict = {"tag":[], "patterns":[]}

for i in range (len(val_data)):
  pattern = val_data[val_data.index==i]['patterns'].values[0]  
  tag = val_data[val_data.index==i]['tag'].values[0]
  for j in range (len(pattern)):
    dict['tag'].append(tag)
    dict['patterns'].append(pattern[j])

val_data = pd.DataFrame.from_dict(dict)
val_data

,tag,patterns
0,qna 1,Apa tujuan utama dari pembentukan Kitab Undang...
1,qna 1,Mengapa diperlukan pembentukan Kitab Undang-Un...
2,qna 1,Apa alasan utama di balik revisi Kitab Undang-...
3,qna 1,Apa yang mendorong pembaruan Kitab Undang-Unda...
4,qna 1,Apa motivasi utama dalam menyusun KUHP yang baru?
...,...,...
277,qna 50,Apa saja contoh kasus pidana yang termasuk ke ...
278,qna 50,Sebutkan beberapa jenis kasus pidana yang diat...
279,qna 50,Bagaimana hukuman dalam kasus pidana khusus di...
280,qna 50,Apa yang membedakan kasus pidana khusus dengan...


In [24]:
# Training
data['tag'].unique()

array(['qna 1', 'qna 2', 'qna 3', 'qna 4', 'qna 5', 'qna 6', 'qna 7',
       'qna 8', 'qna 9', 'qna 10', 'qna 11', 'qna 12', 'qna 13', 'qna 14',
       'qna 15', 'qna 16', 'qna 17', 'qna 18', 'qna 19', 'qna 20',
       'qna 21', 'qna 22', 'qna 23', 'qna 24', 'qna 25', 'qna 26',
       'qna 27', 'qna 28', 'qna 29', 'qna 30', 'qna 31', 'qna 32',
       'qna 33', 'qna 34', 'qna 35', 'qna 36', 'qna 37', 'qna 38',
       'qna 39', 'qna 40', 'qna 41', 'qna 42', 'qna 43', 'qna 44',
       'qna 45', 'qna 46', 'qna 47', 'qna 48', 'qna 49', 'qna 50'],
      dtype=object)

In [25]:
with open('C:/Users/khalf/Code/LegalScope/LegalScopeChatbot/dictionaries.json', 'r') as r:
  json_dict = json.load(r)
tokenizer_json=json_dict.keys()

tokenizer = Tokenizer(oov_token='')
tokenizer.fit_on_texts(tokenizer_json)

word_index = tokenizer.word_index
print(len(word_index))

18727


In [26]:
# !rm -f glove.6B.zip
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [27]:

from pyexpat import model


def chatbot_test():
  while True:
    user_input = input("User: ").lower()
    pattern_test= user_input
    pattern_test = tokenizer.texts_to_sequences([pattern_test])
    x_test = pad_sequences(pattern_test, x_train.shape[1])

    prediction = model.predict(x_test)[0]
    label = label_encode.inverse_transform([np.argmax(prediction)])[0]
    response = data[data['tag'] == label]['responses'].values[0]
    print("Label:",label)
    print("Chatbot: ", format(random.choice(response)))
    if label=='goodbye' or label=='thanks':
      break